# AES-FIR-FFT 

## Initialization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(formatter={'int':hex})
from siha import *

inputFileName = 'data/inputSignal.bin'
High_Pass_Filter = 'data/reload_hpf.bin'
Band_Pass_Filter = 'data/reload_bpf.bin'
Low_Pass_Filter = 'data/reload_lpf.bin'
output = 'data/origSignal.bin'

# FFT Configuration Data
fft_config = np.array([0x00005F0F, 0x00005F0F, 0x00005F0F, 0x00005F0F], dtype=np.uint32)

# FIR Filter Configuration Data
fir_config = np.array([0x0000000c,0x0000000c,0x0000000c,0x0000000c], dtype=np.uint32)

# AES Key Configuration Data
encKeybuff = np.array([
    0x0c0d0e0f, 0x08090a0b, 0x04050607, 0x00010203,
    0x00000001, 0x00000000, 0x00000000, 0x00000000], dtype=np.uint32)
decKeybuff = np.array([
    0x0c0d0e0f, 0x08090a0b, 0x04050607, 0x00010203,
    0x00000000, 0x00000000, 0x00000000, 0x00000000], dtype=np.uint32);

## Encrypted Signal

In [ ]:
BUFFSIZE = 0x4000000
inbuff = np.fromfile(inputFileName, dtype=np.uint32)[:BUFFSIZE]
size = inbuff.shape[0]
inbuff1 = inbuff.astype(np.int16)

# Plot
plt.plot(inbuff1[1000:1500])
plt.xlabel('Time')
plt.ylabel('Amplitude');

## Running AES128 accelerator to decrypt input

In [ ]:
slot_AES128 = 0 # change slot number here if required

In [ ]:
offset_key = 0
offset_input = 64
offset_result = 64
size_config = len(decKeybuff)
tid_conf = 1
tid_run = 0

AESaccel = AcapdAccel("AES", 0)
AESaccel.loadData(decKeybuff, offset_key)
AESaccel.loadData(inbuff, offset_input)
AESaccel.startAccel()
AESaccel.config(offset_key, size_config, tid_conf)
AESaccel.run(offset_input, size, tid_run, offset_result)
outbuff = AESaccel.retrieve(offset_result, size)
origSignal = np.frombuffer(outbuff.tobytes(), dtype=np.uint32)
origSignal.tofile(output)
AESaccel.close();

# Plot
origSignal0 = origSignal.astype(np.int16)
fig = plt.figure(figsize=(10,5))
plt.title('Time Series Signal')
plt.ylabel('Amplitude')
plt.xlabel('Time')
plt.plot(origSignal0[1000:1500]);

## Running FFT accelerator on Decrypted Signal

In [ ]:
slot_FFT = 1 # change slot number here if required

In [ ]:
offset_config = 0
size_config = 1
offset_input = 1024
offset_result = 1024
tid_run = 0

FFTaccel = AcapdAccel("FFT", slot_FFT)
FFTaccel.loadData(fft_config, offset_config)
FFTaccel.loadData(origSignal, offset_input)
FFTaccel.config_fft(offset_config, size_config)
FFTaccel.run(offset_input, size, tid_run, offset_result)
FFTofOrigSignal = FFTaccel.retrieve(offset_result, size)
FFTaccel.close();

# Plot
N = 1024*4
x = np.linspace(0, 10**9/2, int(N/2))
f = np.frombuffer(FFTofOrigSignal.tobytes(), dtype=np.int16)
origSignal0FFT = np.sqrt(np.square(f[0::2].astype(np.float64)) + np.square(f[1::2].astype(np.float64)))
i = 0

fig = plt.figure(figsize=(10,5))
plt.title('FFT')
plt.ylabel('E')
plt.xlabel('Frequency')
plt.plot(x, np.abs(origSignal0FFT[i*N:i*N+int(N/2)]));

## Running FIR accelerator on Decrypted signal

In [ ]:
slot_FIR = 0 # change slot number here if required

In [ ]:
Applied_Filter = 'data/reload_hpf.bin'
fir_reload = np.fromfile(Applied_Filter, dtype=np.uint32)

offset_reload = 0
offset_config = 128
offset_input = 256
offset_output = 256
tid_reload = 1
tid_config = 2
tid_run = 0

FIRaccel = AcapdAccel("FIR", slot_FIR)
FIRaccel.loadData(fir_reload, offset_reload)
FIRaccel.loadData(fir_config, offset_config)
FIRaccel.loadData(origSignal, offset_input)
FIRaccel.config(offset_reload, len(fir_reload), tid_reload)
FIRaccel.config(offset_config, len(fir_config), tid_config)
FIRaccel.run(offset_input, size, tid_run, offset_output)
FilteredSignal = FIRaccel.retrieve(offset_output, size)
FIRaccel.close();

# Plot
FilteredSignal0 = FilteredSignal.astype(np.int16)
fig = plt.figure(figsize=(10,5))
plt.title('Time Series Signal')
plt.ylabel('Amplitude')
plt.xlabel('Time')
plt.plot(FilteredSignal0[1000:1500]);

## Running FFT accelerator on Filtered Signal

In [ ]:
slot_FFT = 1 # change slot number here if required

In [ ]:
offset_config = 0
size_config = 1
offset_input = 1024
offset_result = 1024
tid_run = 0

FFTaccel = AcapdAccel("FFT", slot_FFT)
FFTaccel.loadData(fft_config, offset_config)
FFTaccel.loadData(FilteredSignal, offset_input)
FFTaccel.config_fft(offset_config, size_config)
FFTaccel.run(offset_input, size, tid_run, offset_result)
FFTofFilteredSignal = FFTaccel.retrieve(offset_result, size)
FFTaccel.close();

# Plot
N = 1024*4
x = np.linspace(0, 10**9/2, int(N/2))
f = np.frombuffer(FFTofFilteredSignal.tobytes(), dtype=np.int16)
FilteredSignalFFT = np.square(f[0::2].astype(np.float64)) + np.square(f[1::2].astype(np.float64))
i = 0

fig = plt.figure(figsize=(10,5))
plt.title('FFT')
plt.ylabel('E')
plt.xlabel('Frequency')
plt.plot(x, np.abs(FilteredSignalFFT[i*N:i*N+int(N/2)]));